In [ ]:
from IPython import display 
import pandas as pd
import gmaps

from api_key import gkey

#### Hello Pam!

# Libraries + Happiness
Mallory Anderson, Ellie Bunz, Pam Meyer, & Zoe Poeschl

### Define the core message or hypothesis of your project.
- Are there more libraries in happier countries?
- Does usage of libraries correlate with happiness rankings? How strong is the correlation? 

### Describe the questions you asked, and why you asked them
- What are some of the happiest countries? 
- What % of libraries are public? 
- Are there more libraries in happier countries?
- Does visiting libraries make a happier country? 
- Do the amount of libraries make a happier country? 

# Data
To answer the questions we pulled from a library data database, a happiness data database, and a population database, and pulled capitals lat and lng. These were downloaded to csv and imported. 

## Cleanup and Exploration
After cleaning data and removing NaN we have fewer countries than anticipated for the analysis. In addition, we realized that it would be best to normalize the data. 

In [ ]:
#read csv to make maps
csv_path_map = "Resources/map_data.csv"
csv_path_library_count = "Resources/library_count.csv"
csv_path_visit_count = "Resources/visits_count.csv"

map_df = pd.read_csv(csv_path_map)
lib_amt_df = pd.read_csv(csv_path_library_count)
visits_df = pd.read_csv(csv_path_visit_count)

In [ ]:
map_df.head()

In [ ]:
lib_amt_df.head()

In [ ]:
visits_df.head()

#### Hi Ellie!

## Question: What percent of libraries are public?

### Library visits per library type

In [ ]:
display.Image("./images/VisitsPerLibraryType.png")

### Amount of libraries per library type

In [ ]:
display.Image("./images/AmountOfLibrariesPerLibraryType.png")

## Question: What are some of the happiest countries?

### Top 20 happiest countries

In [ ]:
display.Image("./images/happiest_countries.png")

### Top 20 happiest countries with library usage data

In [ ]:
display.Image("./images/happiest_library_countries.png")

### Heatmap of overall happiness

In [ ]:
# Configure maps
gmaps.configure(api_key=gkey)

fig1 = gmaps.figure(center=(20,5), zoom_level=2)

# Store lat and lng in list
locations = map_df[["CapitalLatitude","CapitalLongitude"]]

# Store happiness to set weight
happiness = map_df["Happiness.Score"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=happiness, dissipating=False, max_intensity=7.6, point_radius=3.5)


# Add heat layer
fig1.add_layer(heat_layer)
fig1

#### Hi Zoe!

## Data Analysis
Discuss the steps you took to analyze the data. 

## Question: Are there more libraries in happier countries? 

### Libraries per country

In [ ]:
display.Image("./images/libraries_per_country_bar.png")

### Libraries per capita

In [ ]:
#Add markers for number of libraries per capita
lib_locations = lib_amt_df[["CapitalLatitude","CapitalLongitude"]].astype(float)

lib_amt_df["Lib Per 100000"] = lib_amt_df["Libraries Per Capita"]*100000
lib_amt_df["Lib Per 100000"] = lib_amt_df["Lib Per 100000"].round(2)

lib_amt_df = lib_amt_df.rename(columns={"Happiness.Rank": "HappinessRank", 
                          "Happiness.Score":"HappinessScore"})

lib_amt_df["HappinessRank"] = lib_amt_df["HappinessRank"].astype(int)


In [ ]:
info_box_template = """
<dl>
<dt>Country</dt><dd>{Country}</dd>
<dt>Happiness Rank</dt><dd>{HappinessRank}</dd>
<dt>Libraries Per 100000 People</dt><dd>{Lib Per 100000}</dd>
</dl>
"""

#<dt>Happiness Rank</dt><dd>{Happiness.Rank}</dd>
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
library_info = [info_box_template.format(**row) for index, row in lib_amt_df.iterrows()]

In [ ]:
fig3 = gmaps.figure(center=(20,30), zoom_level=2)

#Create symbol layer
libraries_layer = gmaps.marker_layer(
    lib_locations, info_box_content=library_info)

fig3.add_layer(heat_layer)
fig3.add_layer(libraries_layer)

fig3

## Question: Does library use make a happier country?

### Number of physical loans versus country happiness scores

In [ ]:
display.Image("./images/physicalloans_v_happiness.png")

### Number of library visits versus country happiness score

In [ ]:
display.Image("./images/physicalvisits_v_happiness.png")

### Count of library visits

In [ ]:
# Add markers for visits 
visit_locations = visits_df[["CapitalLatitude","CapitalLongitude"]].astype(float)
visits = visits_df["Visits"].tolist()

fig2 = gmaps.figure(center=(20,5), zoom_level=2)

#Create symbol layer
visits_layer = gmaps.symbol_layer(
    visit_locations, fill_color='blue',
    stroke_color='rgba(0, 0, 150, 0)', scale=5,
    info_box_content=[f"Visit count: {visit}" for visit in visits]
)

fig2.add_layer(heat_layer)
fig2.add_layer(visits_layer)

fig2

### Library visits per country

In [ ]:
display.Image("./images/visits_per_country.png")

#### Hi Mallory!

## Question: Does checking out materials from libraries make a happier country?

In [ ]:
display.Image("./images/physicalloans_v_happiness_regress.png")

The r-value for this graph is -0.295. This graph tells us that there is a weak correlation between number of physical loans per capita and happiness score. 

## Question: Does visiting libraries make a happier country?

In [ ]:
display.Image("./images/physicalvisits_v_happiness_regress.png")

The r-value for this graph is 0.39. This graph tells us that there is a weak correlation between number of library visits per capita and happiness score. 

# Summary
Describe whether you were able to answer these questions and our findings.

# Post Mortem
Difficulties that arose, and how you dealt with them. 
Additional questions that came up, but we didn't have time to answer.

## Hypothesis

We have two hypotehses -- one about vistiaton and one about count. 
Rank based on visitaiton, divide in thirds, pass in happiness scores for top third and bottom third and see if they're statistically different. We could do a second test with visitiation rates. 

We could come up with a compound score -- library count * visitation -- results in combining these two and giving us a library "score". 

### Hypothesis
If libraries make people happy, then countries with more libraries and higher visitation rates of those libraries will have higher happiness scores.

### Null Hypothesis
If libraries do not generate happiness then there will library count and library visits will have no impact on happiness scores.

### Findings
The p-value for number of libraries versus happiness scores was ...
The p-value for number of visits to libraries versus happiness scores was...

A p-value less than 0.05 means we must reject our null hypothesis. If it is greater we fail to reject our null hypothesis.

ANOVA extends the t-test to multiple categories. 